This novel adversarial attack method that I have developed is called <b>CEIA (Contextual Embedding Inversion Attack)</b>. This method is a sophisticated and novel approach to creating adversarial examples for BERT-based models, particularly targeting question-answering systems.<br /><br />

The CEIA method is a novel and sophisticated adversarial attack that challenges BERT-based question-answering systems in several ways:
<ul>
    <li><b>Semantic Inversion:</b> By finding and applying antonym phrases, it alters the meaning of parts of the context and questions without making them grammatically incorrect or obviously altered.</li>
    <li><b>Contextual Understanding Exploitation:</b> CEIA leverages BERT's own contextual embeddings to create adversarial examples, making it particularly effective against BERT-based models.</li>
    <li><b>Preservation of Key Information:</b> By keeping answer phrases intact, it ensures that the questions remain answerable but requires models to navigate through potentially contradictory information.</li>
    <li><b>Noun Chunk Targeting:</b> The attack focusses on inverting noun chunks, which are often the most informative parts of a sentence, making the attack more impactful.</li>
    <li><b>Robustness Testing:</b> This method effectively tests the robustness of QA models against subtle semantic changes in both context and questions.</li>
</ul>

The CEIA method represents a significant advancement in the field of adversarial attacks on NLP systems. It creates adversarial examples that are semantically coherent but potentially misleading, while preserving grammatical structure. This makes the attack particularly challenging to detect and defend against.<br /><br />

The approach I have developed could be valuable for improving the robustness and reliability of question-answering systems in real-world applications where input texts might contain contradictions or inconsistencies. It also highlights the importance of developing more robust NLP models that can handle semantic inversions and contextual contradictions.

### Import and Model Loading
Imports the necessary libraries and loads spaCy and BERT models.<br />
BERT is crucial for generating contextual embeddings used in the antonym phrase selection process.

In [11]:
import os
import json
import random
import spacy
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from scipy.spatial.distance import cosine

In [2]:
# Load spaCy and BERT models
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
model = BertModel.from_pretrained('bert-base-uncased')

### get_bert_embedding() function
Generates BERT embeddings for a given text, which are essential for the CEIA method.

In [5]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


### find_antonym_phrase() function
This function is a key component of the CEIA attack.<br />
It uses a list of negation words and phrases to find an antonym phrase for a given input.<br />
The function selects the candidate that produces the most semantically different phrase based on cosine distance of BERT embeddings.

In [6]:
def find_antonym_phrase(phrase, threshold=0.6):
    embedding = get_bert_embedding(phrase)
    candidates = [
        "is not", "does not", "was not", "were not", "had not",
        "cannot", "could not", "should not", "would not", "will not",
        "never", "hardly", "barely", "scarcely", "rarely",
        "seldom", "infrequently", "occasionally", "sporadically"
    ]
    
    best_candidate = None
    best_distance = 0
    
    for candidate in candidates:
        new_phrase = f"{candidate} {phrase}"
        new_embedding = get_bert_embedding(new_phrase)
        distance = cosine(embedding, new_embedding)
        
        if distance > best_distance:
            best_distance = distance
            best_candidate = candidate
    
    if best_distance > threshold:
        return f"{best_candidate} {phrase}"
    return phrase


### ceia_attack() function
This is the main implementation of the CEIA attack.<br />
It processes the text sentence by sentence, identifying noun chunks.<br />
For each noun chunk not in the preserve_phrases list, it may apply the antonym inversion based on a random threshold.<br />
This creates a modified version of the text with inverted meanings for some phrases, while preserving overall grammatical structure.

In [7]:
def ceia_attack(text, importance_threshold=0.7, preserve_phrases=None):
    if preserve_phrases is None:
        preserve_phrases = set()
    else:
        preserve_phrases = set(phrase.lower() for phrase in preserve_phrases)
    
    doc = nlp(text)
    sentences = list(doc.sents)
    
    modified_sentences = []
    for sentence in sentences:
        words = [token.text for token in sentence]
        for chunk in sentence.noun_chunks:
            if chunk.text.lower() not in preserve_phrases and random.random() > importance_threshold:
                inverted_chunk = find_antonym_phrase(chunk.text)
                if inverted_chunk != chunk.text:
                    words[chunk.start:chunk.end] = inverted_chunk.split()
        modified_sentences.append(' '.join(words))
    
    return ' '.join(modified_sentences)

### find_answer_position() function
Locates the position of an answer in the modified context, handling potential changes in word positions.

In [8]:
def find_answer_position(context, answer):
    answer_start = context.find(answer)
    if answer_start != -1:
        return answer_start
    
    words = context.split()
    answer_words = answer.split()
    for i in range(len(words) - len(answer_words) + 1):
        if ' '.join(words[i:i+len(answer_words)]).lower() == answer.lower():
            return len(' '.join(words[:i])) + 1
    
    return -1

### process_squad_file() function
Applies the CEIA attack to the SQuAD dataset, modifying both context and questions.<br />
It preserves answer phrases to ensure they remain intact in the modified context.<br />
Updates answer positions in the modified context.

In [9]:
def process_squad_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    new_data = {"version": "v2.0", "data": []}

    for article in tqdm(data['data'], desc="Processing articles"):
        new_article = {"title": article['title'], "paragraphs": []}
        
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            
            answers = set()
            for qa in paragraph['qas']:
                if qa['answers']:
                    answers.add(qa['answers'][0]['text'])
            
            adv_context = ceia_attack(context, preserve_phrases=answers)
            
            new_qas = []
            for qa in paragraph['qas']:
                new_qa = qa.copy()
                
                if qa['answers']:
                    answer = qa['answers'][0]['text']
                    new_qa['question'] = ceia_attack(qa['question'], preserve_phrases=[answer])
                else:
                    new_qa['question'] = ceia_attack(qa['question'])
                
                if new_qa['answers']:
                    answer = new_qa['answers'][0]['text']
                    answer_start = find_answer_position(adv_context, answer)
                    
                    if answer_start != -1:
                        new_qa['answers'][0]['answer_start'] = answer_start
                    else:
                        print(f"Warning: Answer '{answer}' not found in modified context. Keeping original position.")
                
                new_qas.append(new_qa)
            
            new_paragraph = {
                "context": adv_context,
                "qas": new_qas
            }
            new_article['paragraphs'].append(new_paragraph)
        
        new_data['data'].append(new_article)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(new_data, f, ensure_ascii=False, indent=2)


### Main execution
Applies the CEIA attack to the SQuAD v2.0 training set.

In [12]:
# Create data set
path = "SQuAD/"
input_file = os.path.join(path, "train-v2.0.json")
output_file = os.path.join(path, "squad-v2.0-CEIA.json")
process_squad_file(input_file, output_file)

Processing articles:   0%|                              | 0/442 [00:00<?, ?it/s]

Processing articles:   0%|                | 1/442 [21:48<160:20:16, 1308.88s/it]

Processing articles:   0%|                | 2/442 [42:46<156:18:28, 1278.88s/it]

Processing articles:   1%|                | 3/442 [58:01<135:42:01, 1112.81s/it]

Processing articles:   1%|▏              | 4/442 [1:08:46<112:54:13, 927.98s/it]

...

Processing articles:  99%|███████████████▊| 436/442 [71:53:22<56:05, 560.96s/it]

Processing articles:  99%|███████████████▊| 437/442 [72:05:29<50:53, 610.78s/it]

Processing articles:  99%|███████████████▊| 438/442 [72:14:21<39:09, 587.27s/it]

Processing articles:  99%|███████████████▉| 439/442 [72:25:37<30:41, 613.91s/it]

Processing articles: 100%|████████████████| 442/442 [72:49:44<00:00, 593.18s/it]
